<a href="https://colab.research.google.com/github/laffertybrian/W_William_Project_001/blob/main/Genetic_Variant_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Libraries
import pandas as pd
import numpy as np

In [11]:
# Loading dataset

df = pd.read_csv('/content/drive/MyDrive/Data Science Projects/clinvar_conflicting.csv', dtype={'CHROM': str, 38: str, 40: object})
df.head()

,CHROM,POS,REF,ALT,AF_ESP,AF_EXAC,AF_TGP,CLNDISDB,CLNDISDBINCL,CLNDN,...,SIFT,PolyPhen,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,LoFtool,CADD_PHRED,CADD_RAW,BLOSUM62
0,1,1168180,G,C,0.0771,0.10020,0.1066,MedGen:CN169374,NaN,not_specified,...,tolerated,benign,NaN,NaN,NaN,NaN,NaN,1.053,-0.208682,2.0
1,1,1470752,G,A,0.0000,0.00000,0.0000,"MedGen:C1843891,OMIM:607454,Orphanet:ORPHA9877...",NaN,Spinocerebellar_ataxia_21|not_provided,...,deleterious_low_confidence,benign,NaN,NaN,NaN,NaN,NaN,31.000,6.517838,-3.0
2,1,1737942,A,G,0.0000,0.00001,0.0000,"Human_Phenotype_Ontology:HP:0000486,MedGen:C00...",NaN,Strabismus|Nystagmus|Hypothyroidism|Intellectu...,...,deleterious,probably_damaging,NaN,NaN,NaN,NaN,NaN,28.100,6.061752,-1.0
3,1,2160305,G,A,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002...",NaN,Shprintzen-Goldberg_syndrome|not_provided,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.500,3.114491,NaN
4,1,2160305,G,T,0.0000,0.00000,0.0000,"MedGen:C1321551,OMIM:182212,SNOMED_CT:83092002",NaN,Shprintzen-Goldberg_syndrome,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.700,4.766224,-3.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65188 entries, 0 to 65187
Data columns (total 46 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CHROM               65188 non-null  object 
 1   POS                 65188 non-null  int64  
 2   REF                 65188 non-null  object 
 3   ALT                 65188 non-null  object 
 4   AF_ESP              65188 non-null  float64
 5   AF_EXAC             65188 non-null  float64
 6   AF_TGP              65188 non-null  float64
 7   CLNDISDB            65188 non-null  object 
 8   CLNDISDBINCL        167 non-null    object 
 9   CLNDN               65188 non-null  object 
 10  CLNDNINCL           167 non-null    object 
 11  CLNHGVS             65188 non-null  object 
 12  CLNSIGINCL          167 non-null    object 
 13  CLNVC               65188 non-null  object 
 14  CLNVI               27659 non-null  object 
 15  MC                  64342 non-null  object 
 16  ORIG

In [58]:
def dataframe_statistics(DF):
  """Takes input of a dataframe and retuns a summary dataframe of key statistics"""

  # initialize lists for data storage
  feature_list = []
  dtype_list = []
  values_list = []
  missing_values_list = []
  data_percent_list = []
  unique_values_list = []
  number_unique_values_list = []

  for column in DF.columns:

    # length and missing values of column
    entries = len(DF[column])
    missing = DF[column].isna().sum()

    feature_list.append(column)
    dtype_list.append(DF[column].dtype)
    values_list.append(entries - missing)
    missing_values_list.append(missing)
    data_percent_list.append(round((entries - missing) / entries, 2))
    unique_values_list.append(DF[column].unique())
    number_unique_values_list.append(len(DF[column].value_counts()))

  # creating summary dataframe for return
  summary = pd.DataFrame({'Features': feature_list, 'Dtype': dtype_list, 
                          '#_Values': values_list, '#_Missing_Values': missing_values_list, 
                          '%_Data': data_percent_list, 'Unique_Values': unique_values_list,
                          '#_Unique_Values': number_unique_values_list})
  
  summary = summary.set_index('Features')
  

  return summary.sort_values(by = '#_Missing_Values', ascending = False)

In [59]:
df_summary = dataframe_statistics(df)
df_summary

,Dtype,#_Values,#_Missing_Values,%_Data,Unique_Values,#_Unique_Values
Features,,,,,,
MOTIF_SCORE_CHANGE,float64,2,65186,0.00,"[nan, -0.063, -0.097]",2
HIGH_INF_POS,object,2,65186,0.00,"[nan, N]",1
MOTIF_POS,float64,2,65186,0.00,"[nan, 1.0]",1
MOTIF_NAME,object,2,65186,0.00,"[nan, Egr1:MA0341.1, FOXA1:MA0546.1]",2
DISTANCE,float64,108,65080,0.00,"[nan, 1811.0, 1855.0, 2202.0, 1651.0, 1407.0, ...",96
SSR,float64,130,65058,0.00,"[nan, 1.0, 16.0]",2
CLNSIGINCL,object,167,65021,0.00,"[nan, 424754:Likely_pathogenic, 30118:risk_fac...",137
CLNDISDBINCL,object,167,65021,0.00,"[nan, MedGen:C1828210,OMIM:153870,Orphanet:ORP...",93
CLNDNINCL,object,167,65021,0.00,"[nan, Bull's_eye_maculopathy|Methylmalonic_aci...",101


In [54]:
df['Allele'].value_counts()

T                      19991
A                      19800
G                      11397
C                      10761
-                       2510
                       ...  
TCTTCTGCTGCTTCTT           1
AATATTTGCCTC               1
GAAAAATACAGCTTCCTCT        1
AGCAAGTTGGGGTGTGC          1
CGA                        1
Name: Allele, Length: 374, dtype: int64